In [1]:
#-*- coding:utf-8
import sys
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

import GameEnv
import random
import numpy as np
import DQN

gameEnv = GameEnv.GlodenFlower([20000,20000])
RLModel = DQN.DQN(embedding_size=10,sequence_length=20,learning_rate=0.1,batch_size=500)
gameEnv.debug = False
memory = []
for episode in range(500000):
    # 初始化环境
    gameEnv.reset()

    playerI = gameEnv.getStartTurn()
    observation_this = [[],gameEnv.playerCards["A"],gameEnv.personStatus["A"]]
    if playerI == "B":
        availble_actions = gameEnv.chooseAvailbleAction(playerI)
        action = RLModel.choose_action(np.array([observation_this]),availble_actions)
        #print (playerI, action, gameEnv.deskMoney, gameEnv.nowPrice)
        observation_next, reward, done = gameEnv.step(action, "B")
        playerI = "A"
        if done:
            continue

    while True:
        # DQN 根据观测值选择行为
        availble_actions = gameEnv.chooseAvailbleAction(playerI)
        action = RLModel.choose_action(np.array([observation_this]),availble_actions)
        # 环境根据行为给出下一个 state, reward, 是否终止
        #print (playerI, action, gameEnv.deskMoney, gameEnv.nowPrice,gameEnv.personStatus[playerI])
        observation_next, reward, done = gameEnv.stepA(action,RLModel)


        # DQN 存储记忆
        # RL.store_transition(observation, action, reward)
        RLModel.store_transition(observation_this, action, reward,done,observation_next)

        # 控制学习起始时间和频率 (先累积一些记忆再开始学习)
        if (episode > 2000) and (episode % 100 == 0):
            if episode % 10000 == 0 :print "episode",episode
            RLModel.train()

        # 将下一个 state_ 变为 下次循环的 state
        observation_this = observation_next

        # 如果终止, 就跳出循环
        if done:
            break

        # end of game
print('game over')



Instructions for updating:
Use `argmax` instead


/opt/anaconda2/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
DQN.py:149: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if action_index not in availble_actions:


('loss', 100, 85.90619)
episode 10000
('loss', 200, 75.87521)
episode 20000
('loss', 300, 81.20457)
episode 30000
episode 30000
('loss', 400, 95.88106)
episode 40000
episode 40000
episode 40000
('loss', 500, 104.50111)
('loss', 600, 66.29911)
episode 50000
episode 50000
episode 50000
('loss', 700, 115.61976)
episode 60000
('loss', 800, 109.79851)
('loss', 900, 91.02393)
episode 70000
episode 70000
('loss', 1000, 81.107346)
episode 80000
('loss', 1100, 85.606766)
episode 90000
episode 90000
('loss', 1200, 100.627785)
('loss', 1300, 96.24274)
episode 100000
('loss', 1400, 102.49254)
episode 110000
('loss', 1500, 79.0897)
episode 120000
('loss', 1600, 125.731346)
('loss', 1700, 85.38411)
episode 130000
('loss', 1800, 83.984245)
('loss', 1900, 104.477165)
episode 150000
episode 150000
('loss', 2000, 64.22027)
('loss', 2100, 76.53368)
('loss', 2200, 88.08682)
episode 170000
('loss', 2300, 113.84502)
('loss', 2400, 97.78568)
episode 180000
episode 180000
('loss', 2500, 78.586975)
episode 190

KeyboardInterrupt: 

In [2]:
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array([[["A_看_5", "B_看_5", "A_看_5", "B_看_20"], ['diamond_7', 'heart_7', 'spade_7'],"看"]])))
for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_reverse_index_dicts[i],v

0 闷_2 0.0
1 闷_4 0.0
2 闷_8 0.0
3 闷开_0 0.0
4 看_2 2.6749284
5 看_5 2.5375712
6 看_10 2.2168646
7 看_20 1.5940257
8 开_0 -7.2232695
9 丢_0 -7.316115


In [3]:
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array([[["A_闷_8", "B_闷_8", "A_闷_8", "B_看_20"], ['diamond_7', 'heart_7', 'spade_7'],"闷"]])))
for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_reverse_index_dicts[i],v

0 闷_2 0.8323411
1 闷_4 0.80202866
2 闷_8 0.31619662
3 闷开_0 0.66839844
4 看_2 0.0
5 看_5 0.0
6 看_10 0.0
7 看_20 0.0
8 开_0 -0.0
9 丢_0 -0.0


In [7]:
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array([[[], ['diamond_2', 'heart_2', 'spade_2'],"看"]])))
for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_reverse_index_dicts[i],v

0 闷_2 0.0
1 闷_4 0.0
2 闷_8 0.0
3 闷开_0 0.0
4 看_2 1.7622914
5 看_5 1.1329337
6 看_10 1.0658267
7 看_20 0.98763716
8 开_0 -3.502627
9 丢_0 -3.6997738


In [10]:
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array([[[], ['diamond_4', 'heart_5', 'spade_2'],"看"]])))
for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_reverse_index_dicts[i],v

0 闷_2 0.0
1 闷_4 0.0
2 闷_8 0.0
3 闷开_0 0.0
4 看_2 0.8531427
5 看_5 2.3518207
6 看_10 1.4212455
7 看_20 2.0196583
8 开_0 -1.1234959
9 丢_0 -4.4069185


In [11]:
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array([[[], ['diamond_1', 'heart_5', 'spade_2'],"闷"]])))
for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_reverse_index_dicts[i],v

0 闷_2 0.95227855
1 闷_4 1.1853846
2 闷_8 0.81203157
3 闷开_0 0.95211285
4 看_2 0.0
5 看_5 0.0
6 看_10 0.0
7 看_20 0.0
8 开_0 -0.0
9 丢_0 -0.0


In [12]:
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array([[["A_闷_4", "B_看_20","A_闷_4", "B_看_20"], ['diamond_2', 'heart_2', 'spade_2'],"闷"]])))
for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_reverse_index_dicts[i],v

0 闷_2 1.29885
1 闷_4 1.3534505
2 闷_8 0.9347714
3 闷开_0 1.4973941
4 看_2 0.0
5 看_5 0.0
6 看_10 0.0
7 看_20 0.0
8 开_0 -0.0
9 丢_0 -0.0


In [14]:
RLModel.sess.run(RLModel.predictionsNotSee,feed_dict=RLModel._feed_dict(np.array([[["A_闷_4"], ['diamond_2', 'heart_2', 'spade_2'],"闷"]])))

array([[ 0.95227855,  1.1853846 ,  0.81203157,  0.95211285, -0.19811791]],
      dtype=float32)

In [15]:
RLModel.sess.run(RLModel.playSequenceLengthInput,feed_dict=RLModel._feed_dict(np.array([[["A_闷_8"], ['diamond_1', 'heart_2', 'spade_3'],"闷"]])))

array([2], dtype=int32)

In [17]:
RLModel.sess.run(RLModel.last_output,feed_dict=RLModel._feed_dict(np.array([[["A_闷_8"], ['diamond_1', 'heart_2', 'spade_3'],"闷"]])))

array([[-0.5002861 , -0.04074519,  0.44523266, -0.11365644,  0.55178744,
         0.23713528,  0.4065764 ,  0.37365475,  0.05763792,  0.5122157 ]],
      dtype=float32)

In [4]:
x = RLModel.experience_replay()

In [9]:
len(x[x[:,2] != 0])

182

In [10]:
len(x)

500